# Pyspark code Testing - 6 
# MLib


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('ML').getOrCreate()

23/12/18 21:07:43 WARN Utils: Your hostname, Soumobratas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
23/12/18 21:07:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/18 21:07:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/18 21:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/18 21:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/12/18 21:07:44 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
training = spark.read.csv('sample_dataframe.csv',header=True,inferSchema=True)

In [4]:
training.show()

+---------+---+---------+---+------+
|     Name|Age|     City|Exp|salary|
+---------+---+---------+---+------+
|    Soumo| 23|Bangalore|  2|  1000|
|  Subrata| 53|  Chennai|  3|  2000|
|Anindita | 50|  Chennai|  2|  3000|
|  Purnima| 75|Bangalore|  4|   800|
|      Uma| 80|  Chennai|  5|   600|
+---------+---+---------+---+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Exp: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'City', 'Exp', 'salary']

['Age','Exp'] -------> new feature ------> Independent Feature

In [11]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['Age','Exp'],outputCol='Independent Feature')

In [12]:
output = featureassembler.transform(training)

In [13]:
output.show()

+---------+---+---------+---+------+-------------------+
|     Name|Age|     City|Exp|salary|Independent Feature|
+---------+---+---------+---+------+-------------------+
|    Soumo| 23|Bangalore|  2|  1000|         [23.0,2.0]|
|  Subrata| 53|  Chennai|  3|  2000|         [53.0,3.0]|
|Anindita | 50|  Chennai|  2|  3000|         [50.0,2.0]|
|  Purnima| 75|Bangalore|  4|   800|         [75.0,4.0]|
|      Uma| 80|  Chennai|  5|   600|         [80.0,5.0]|
+---------+---+---------+---+------+-------------------+



In [14]:
output.columns

['Name', 'Age', 'City', 'Exp', 'salary', 'Independent Feature']

In [15]:
finalized_data = output.select("Independent Feature",'salary')

In [16]:
finalized_data.show()

+-------------------+------+
|Independent Feature|salary|
+-------------------+------+
|         [23.0,2.0]|  1000|
|         [53.0,3.0]|  2000|
|         [50.0,2.0]|  3000|
|         [75.0,4.0]|   800|
|         [80.0,5.0]|   600|
+-------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
#train test split
# 0.75 will contain training data and the remaining 0.25 will contain the test data
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Feature',labelCol='salary')
regressor = regressor.fit(train_data)

Exception ignored in: <function JavaWrapper.__del__ at 0x11834d670>
Traceback (most recent call last):
  File "/Users/soumobratamanna/miniforge3/lib/python3.9/site-packages/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x11834d670>
Traceback (most recent call last):
  File "/Users/soumobratamanna/miniforge3/lib/python3.9/site-packages/pyspark/ml/wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
23/12/18 21:45:54 WARN Instrumentation: [85dfd7d7] regParam is zero, which might cause numerical instability and overfitting.
23/12/18 21:45:55 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/18 21:45:55 WARN InstanceBuilder: Faile

Coefficients

In [19]:
regressor.coefficients

DenseVector([59.4917, -1437.3849])

Intercepts

In [21]:
regressor.intercept

2736.2005679063595

Prediction

In [25]:
#pred_results = regressor.evaluate(test_data)

In [26]:
#pred_results.predictions.show()